# Imports

In [1]:
using JuMP, GLPK
using VoronoiCells, GeometryBasics, Plots
using Random
using Statistics
using Distributions
using StatsBase

ArgumentError: ArgumentError: Package VoronoiCells [e3e34ffb-84e9-5012-9490-92c94d0c60a4] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


# Funciones

In [2]:
function get_cost_matrix(x, y, norm=2)    
    m = size(x)[1]
    n = size(y)[1]
    #Calculo los costos Cij = |xi-yj| (distancia euclidea en R^2)
    c1 = (x[:,1]*ones(1,n) - ones(m,1)*y[:,1]').^norm
    c2 = (x[:,2]*ones(1,n) - ones(m,1)*y[:,2]').^norm
    return (c1+c2).^(1/norm)
end

function plot_locations(x, title)
    
    gr()
    Plots.GRBackend()
    
    m = size(x)[1]
    
    X = x[:,1]
    Y = x[:,2]
    
    scatter(X, Y, aspect_ratio=:equal, markersize = 3, xlim = (-0.1, 1.8), ylim = (-0.1, 1.8), title = title, legend = false)
    
end

function plot_stations(y, title)
   
    plot_locations(y, title)
    
    n = size(y)[1];
    
    points = [Point2(y[i,1], y[i,2]) for i in 1:n];
    
    scatter!(y[:,1], y[:,2], markersize = 6, label = "Stations", markershape = :star) # Plot Stations
    annotate!([(points[i][1] + 0.02, points[i][2] + 0.03, text(i, :black)) for i in 1:n]) # Add Station Number
    
end

function plot_assignments(x, y, Pi, voronoi = true)
    gr()
    Plots.GRBackend()

    # Define m and n
    m = size(x)[1]
    n = size(y)[1]

    # Create Stations List
    points = [Point2(y[i,1], y[i,2]) for i in 1:n] 

    # Voronoi Cells
    rect = Rectangle(Point2(0.0, 0.0), Point2(1.5, 1.5))
    tess = voronoicells(points, rect);
        
    if voronoi == true
        plot(tess, color = :green, alpha = 0.4)
    end

    #Assigments
    assigned_index = [argmax(Pi[i,:]) for i in 1:m];
    
    #Array of arrays. Each array is the amount of cars that go on a particular station
    X = [ x[findall(x->x==i, assigned_index),1] for i in 1:n];
    Y = [ x[findall(x->x==i, assigned_index),2] for i in 1:n];

    for i = 1:n
        scatter!(X[i], Y[i], aspect_ratio=:equal, markersize = 3, legend = false, labels = "Assigned to $i") # Plot EVs (differente color by each Station)
    end
        
    for i=1:m
        plot!([x[i,1],y[assigned_index[i],1]],[x[i,2],y[assigned_index[i],2]],color=:black, alpha = 0.2, labels = "") 
    end
    
    scatter!(y[:,1], y[:,2], markersize = 8, label = "Stations", markershape = :star, color=:white) # Plot Stations
    annotate!([(points[i][1] + 0.04, points[i][2] + 0.00, text(i, :black)) for i in 1:n]) # Add Station Number
end

function delete_row(matrix, row_index)
    
    return matrix[setdiff(1:end, row_index), :];
    
end

function random_points(size, dx, dy)
   
    rand_x = rand(dx, (size,1))
    rand_y = rand(dy, (size,1))
    
    random_point = hcat(rand_x, rand_y);
    
    return random_point;
    
end

function Geneterate_Ev_distribution(m, congestion_centers)
    
    #Random.seed!(40)

    x = [0 0]
    
    for congestion_point in congestion_centers
    
        mean_x = congestion_point[1];
        mean_y = congestion_point[2];
        variance_x = congestion_point[3];
        variance_y = congestion_point[4];
        amount_of_evs = m * (congestion_point[5] / 100);
        amount_of_evs = floor(amount_of_evs);
        amount_of_evs = Int(amount_of_evs);
        
        dx = Distributions.Normal(mean_x, variance_x)
        dy = Distributions.Normal(mean_y, variance_y)

        new_points = random_points(amount_of_evs, dx, dy)

        x = vcat(x, new_points)  

    end

    x = delete_row(x, 1);
    
    return sortslices(x, dims = 1);
    
end

function Solve_Model(C, s_max)
    
    m = size(C)[1];
    n = size(C)[2];
    
    q = ones(m,1);
   
    model = JuMP.Model(GLPK.Optimizer)
        
    @variable(model,Pi[1:m,1:n]>=0)
    
    p = sum(Pi, dims=2);
    s = sum(Pi, dims=1);

    @constraint(model, p .== q)
    @constraint(model, con, s .- s_max .<= zeros(1,n)) 

    @objective(model, Min, sum( C.*Pi ))

    optimize!(model)
    
    Pi = value.(Pi) 

    s = sum(Pi, dims = 1);
   
    mu = dual.(con)
    return s, abs.(mu), Pi;
    
end

function Print_Model(C, s_max)
    
    m = size(C)[1];
    n = size(C)[2];
    
    q = ones(m,1);
   
    model = JuMP.Model(GLPK.Optimizer)
        
    @variable(model,Pi[1:m,1:n]>=0)
    
    p = sum(Pi, dims=2);
    s = sum(Pi, dims=1);

    @constraint(model, p .== q)
    @constraint(model, con, s .- s_max .<= zeros(1,n)) 

    @objective(model, Min, sum( C.*Pi ))

    optimize!(model)   
    
    println(dual.(con))

    Pi = value.(Pi) 
    
    plot_assignments(x, y, Pi)

    
end

struct Arrival
    stations :: Vector{Float64}
    ArrivalTime::Float64
    ChargeTime::Float64
end

function get_charged_stations_indexes(arrivals, t)
    
    # Finds all indexes of rows that arrival time + charged time is greater than
    # actual time
    return findall(arrival -> t >= arrival.ArrivalTime + arrival.ChargeTime, arrivals)
end

function has_any_car_been_charged(s_charged_indexes)
    return size(s_charged_indexes)[1] > 0
end

function convert_vector_to_matrix(vector)
    return reduce(hcat,vector)';
end

function create_arrival(s, t, dist)
   charge_time = rand(dist)
   charge_time = round(charge_time, digits=1)
   return Arrival(s[1,:], t, charge_time)
end

function get_stations(arrivals)
    return map(arrival ->arrival.stations, arrivals)
end

function get_total_stations_assignment(arrivals)
    stations = get_stations(arrivals)
    stations_total = sum(stations, dims = 1)
    stations_total = convert_vector_to_matrix(stations_total)
    return stations_total
end

function probability_of_charged(t)
    a = 1/200
    p = 1 - exp(-a*t)
    return p
end

function has_car_charged(t)
    p = probability_of_charged(t)
    n = rand()
    return n < p  
end

function plot_per_station(time, stations_per_time, ylabel)
    number_of_stations = length(stations_per_time[1])
    
    attribute_per_stations = []

    for station_number = 1:number_of_stations
        attribute_of_station = map(attribute -> attribute[station_number], stations_per_time)
        push!(attribute_per_stations, attribute_of_station)
    end

    plot(time, attribute_per_stations, xlabel="Time (s)", ylabel=ylabel)
end

plot_per_station (generic function with 1 method)

# EVs demand distribution

In [3]:
congestion_centers = [(0.6, 0.5, 0.08, 0.15, 70), (0.4, 0.4, 0.13, 0.05, 30)];

m = 300;

x = Geneterate_Ev_distribution(m, congestion_centers);

plot_locations(x, "Evs demand distribution")

UndefVarError: UndefVarError: Distributions not defined

# Station locations

In [4]:
Random.seed!(23)

d = Distributions.Normal(0.5, 0.22)

n = 4;

y_stations = sortslices(rand(d, (n-1,2)), dims = 1);

far = [1.5 1.5]

y = [y_stations; far]

plot_stations(y, "Station locations")
savefig("stations.png")

UndefVarError: UndefVarError: Random not defined

# Dynamic run

In [5]:
# Cars
m = 100;
norm = 2;
congestion_centers = [(0.6, 0.5, 0.1, 0.15, 65), (0.4, 0.3, 0.13, 0.12, 35)];

# Time
t = 0;
k = 5;
dt = 1;
t_final = 200;

# Distributions
dist_arrivals = Poisson(50)
dist_charges = Poisson(50)

charge_time = 50;

# First arrival time
previous_arrival_time = t;
arrival_time = rand(dist_arrivals);

arrivals = Arrival[];

arrival_times = [];
occupations = [];
T = [];
mus = [];

# Stations max capacity
stations_capacity_ratios =  ones(1, n);

# stations_capacity_ratios[1] = 0;
stations_capacity_ratios[n] = 2;

max_capacity = stations_capacity_ratios .* 250

free_charges = max_capacity;
busy_chargers = zeros(1,n);


while t <= t_final
    
    # arrival happened
    if t % k == 0
        
        # Departures
        i = 1;
        departures = zeros(1,n);
        for station in busy_chargers
            dist = Binomial(station, k / charge_time);
            departure = rand(dist);
            departures[i] = departure;
            i += 1
        end
                
        free_charges = free_charges + departures;
        busy_chargers = max_capacity .- free_charges;
        
        # println("t: $t | Departure $departures |  Busy chargers $busy_chargers");
        
        m = rand(dist_arrivals);
        m = floor(m);
                
        x = Geneterate_Ev_distribution(m, congestion_centers);
        C = get_cost_matrix(x, y, norm);
        
        s, mu, Pi = Solve_Model(C , free_charges);
        
        free_charges = free_charges - s;
        busy_chargers = max_capacity .- free_charges;
        
        push!(arrival_times, t)
        push!(mus,mu)
        
        # println("t: $t | Arrival $s | Busy chargers $busy_chargers");
                
    end
    
    occupation = max_capacity .- free_charges;
    push!(occupations, occupation)    
    push!(T,t);
    t = t + dt;
          
end

println("Simulation ended");

UndefVarError: UndefVarError: Poisson not defined

# Station Mus in time

In [6]:
plot_per_station(arrival_times, mus, "mu")
savefig("mu.png")

UndefVarError: UndefVarError: arrival_times not defined

# Station occupation in time

In [7]:
plot_per_station(T, occupations, "cars in station")
savefig("occupation.png")

UndefVarError: UndefVarError: T not defined